In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext("local", "spark-task")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/18 21:23:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/18 21:23:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Task-1 (a)

In [3]:
sparkList = list(range(0,40001))

In [4]:
type(sparkList[0])

int

In [5]:
rdd = sc.parallelize(sparkList)

In [6]:
odd_rdd = rdd.filter(lambda x: x%2 == 1)

In [7]:
odd_rdd.take(7)

[1, 3, 5, 7, 9, 11, 13]

## Task-1(b)

In [8]:
import random

In [9]:
random_list = [random.randint(1,100) for _ in range(10000)]

In [10]:
random_rdd = sc.parallelize(random_list, 5)

In [11]:
def bucket(num):

    if 1<= num <= 20:
        return "1-20"
    elif 21<= num <= 40:
        return "21-40"
    elif 41<= num <= 60:
        return "41-60"
    elif 61<= num <= 80:
        return "61-80"
    elif 81<= num <= 100:
        return "81-100"
    else:
        return "OutOfRange"

In [12]:
counts = (random_rdd.map(lambda x: (bucket(x),1)).reduceByKey(lambda a,b: a+b))

In [13]:
res = counts.collect()

In [14]:
for bucket_name, count in sorted(res):
    print(f"({bucket_name}, {count})")

(1-20, 1948)
(21-40, 2037)
(41-60, 1979)
(61-80, 1977)
(81-100, 2059)


## Task-1 (c):

## Task-1(d)

In [23]:
import re

In [15]:
text_rdd = sc.textFile("Text-data.txt")

In [24]:
words_rdd = text_rdd.flatMap(lambda line: re.findall(r'\b\w+\b', line.lower())) # Split each line into words --> no punctuation, lower case

In [25]:
# I need word-pairs. Use map function

words_pairs = words_rdd.map(lambda x: (x,1))

In [26]:
# sum to same words. Use ReduceByKey()

same_words = words_pairs.reduceByKey(lambda a,b: a+b)

In [28]:
# 25 most common words
top_25 = same_words.takeOrdered(25, key=lambda x: -x[1]) 
# If we didn't delete the punctuation mark, this line would give an error.

In [31]:
for word, count in top_25:
    print((word, count))

('is', 144276)
('the', 78015)
('a', 77125)
('of', 70999)
('to', 43196)
('or', 35443)
('in', 31537)
('and', 27278)
('an', 14550)
('that', 13839)
('with', 13356)
('by', 11357)
('for', 11048)
('on', 7501)
('as', 6783)
('from', 6782)
('who', 6003)
('having', 5808)
('he', 4824)
('used', 4803)
('was', 4359)
('his', 3990)
('one', 3808)
('s', 3683)
('at', 3630)


## Task-1 (e)

In [32]:
# set stopwords

stop_words = {
    "a", "an", "the", "and", "or", "but", "if", "while", "because", "as", "until", "of",
    "at", "by", "for", "with", "about", "against", "between", "into", "through", "during",
    "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off",
    "over", "under", "again", "further", "then", "once", "here", "there", "when", "where",
    "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some",
    "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very",
    "can", "will", "just", "don", "should", "now", "is", "am", "are", "was", "were",
    "be", "been", "being", "have", "has", "had", "do", "does", "did", "would", "could",
    "may", "might", "must", "shall", "also", "who", "whom", "this", "that", "these", "those", "you"
}

filtered_words = words_rdd.filter(lambda x: x not in stop_words)

In [33]:
meaningful_words_count = filtered_words.map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b)

In [34]:
top_25_meaningful_words = meaningful_words_count.takeOrdered(25, key=lambda x: -x[1])

In [35]:
for meaningful_word, count in top_25_meaningful_words:
    print((meaningful_word, count))

('having', 5808)
('he', 4824)
('used', 4803)
('his', 3990)
('one', 3808)
('s', 3683)
('small', 3145)
('genus', 3121)
('which', 2835)
('united', 2789)
('states', 2749)
('relating', 2666)
('it', 2613)
('something', 2335)
('large', 2223)
('person', 2214)
('she', 2189)
('manner', 1968)
('her', 1966)
('two', 1866)
('made', 1843)
('act', 1822)
('someone', 1812)
('its', 1731)
('north', 1666)


In [36]:
sc.stop()